In [147]:
import ijson
import pandas as pd
import numpy as np

In [149]:
with open('./hotels.json', 'rb') as input_file:
    columns=list(ijson.items(input_file, 'fields.item'))
    fields=list()
    for item in columns:
        fields.append(item['label'])
    print(fields)

['HOTEL NAME', 'ADDRESS', 'STATE', 'PHONE', 'FAX', 'EMAIL ID', 'WEBSITE', 'TYPE', 'Rooms']


In [171]:
with open('./hotels.json', 'rb') as input_file:
    parser = ijson.parse(input_file)
    row_count=0
    column_count=0
    chunk_count=0
    chunk_size=100
    temp_list=[]
    df=pd.DataFrame(columns=fields)
    result=pd.DataFrame(columns=['HOTEL NAME','STATE','TYPE','Rooms'])
    for prefix, event, value in parser:
        if row_count<chunk_size:
            if(prefix=='data.item.item'):
                if(not column_count == len(fields)-1):
                    temp_list.append(value)
                    column_count+=1
                else:
                    temp_list.append(value)
                    df = df.append(pd.Series(temp_list, index=fields), ignore_index=True)
                    temp_list=[]
                    column_count=0
                    row_count+=1
        else:
            df['Rooms']=df['Rooms'].astype('int')
            row_count=0
            result_idx=df.groupby(['TYPE'])['Rooms'].transform(max) == df['Rooms']
            chunk_result=df[result_idx].reset_index()
            chunk_result=chunk_result[['HOTEL NAME','STATE','TYPE','Rooms']]
            result=result.append(chunk_result,ignore_index=True)
            df.drop(df.index, inplace=True)
            chunk_count+=1
            print('processed {} number of rows'.format(chunk_count*chunk_size))

processed 100 number of rows
processed 200 number of rows
processed 300 number of rows


In [169]:
df['Rooms']=df['Rooms'].astype('int')
row_count=0
result_idx=df.groupby(['TYPE'])['Rooms'].transform(max) == df['Rooms']
chunk_result=df[result_idx].reset_index()
chunk_result=chunk_result[['HOTEL NAME','STATE','TYPE','Rooms']]
result=result.append(chunk_result,ignore_index=True)
df.drop(df.index, inplace=True)
chunk_result.head()

,HOTEL NAME,STATE,TYPE,Rooms
0,The Savera Hotel,TAMIL NADU,4 Star,230


In [170]:
result.head()

,HOTEL NAME,STATE,TYPE,Rooms
0,The Lalit Grand Palace,JAMMU AND KASHMIR,Heritage Grand,112
1,Noor Us Sabah Palace,MADHYA PRADESH,Heritage Classic,57
2,Jehan Numa Palace Hotel,MADHYA PRADESH,Heritage Basic,98
3,The Ashok,DELHI,5 Star Deluxe,550
4,The Taj Mahal Palace & Tower,MAHARASHTRA,5 Star Deluxe,565


In [178]:
result_idx=result.groupby(['TYPE'])['Rooms'].transform(max) == result['Rooms']
final_result=result[result_idx].reset_index()

In [179]:
final_result.shape

(6, 5)

In [180]:
final_result.head(6)

,index,HOTEL NAME,STATE,TYPE,Rooms
0,0,The Lalit Grand Palace,JAMMU AND KASHMIR,Heritage Grand,112
1,1,Noor Us Sabah Palace,MADHYA PRADESH,Heritage Classic,57
2,2,Jehan Numa Palace Hotel,MADHYA PRADESH,Heritage Basic,98
3,4,The Taj Mahal Palace & Tower,MAHARASHTRA,5 Star Deluxe,565
4,5,Novotel Hyderabad Airport,ANDHRA PRADESH,5 Star,305
5,7,Hotel Trident,MAHARASHTRA,4 Star,436
